<h1 style="text-align:center;">ESCUELA POLITÉCNICA NACIONAL</h1>
<h2 style="text-align:center;">FACULTAD DE INGENIERÍA DE SISTEMAS</h2>
<h3 style="text-align:center;">INGENIERÍA EN CIENCIAS DE LA COMPUTACIÓN</h3>

**PERÍODO ACADÉMICO:** 2025-B <br> 
**ASIGNATURA:** ICCD412 Métodos Numéricos  
**GRUPO:** 1 <br>
**PARALELO:** GR2CC <br>
**TIPO DE INSTRUMENTO:** Proyecto - Predicción del crecimiento de ventas de un producto <br>
**ALUMNOS:** Aguilar Renato, Bolaños Juan, Chávez Carlos

### Filtrado de datos

#### 1. Librerías a usar

In [2]:
import pandas as pd

#### 2. Eliminación de columnas innecesarias y filtrado de la familia 'BEVERAGES'

In [4]:
datos_entrada = "train.csv"
datos_filtrados = "beverages.csv"

def filtrarDatos(entrada=datos_entrada, salida=datos_filtrados):
    """
    Lee un archivo CSV en busca de las columnas a usar, en caso de encontrarlas y validarlas,
    guarda esas columnas y elimina los registros que no pertenecen a la familia 'BEVERAGES' para
    almacenarlos en un nuevo CSV.
    param:
    entrada: .csv inicial (dataset principal del que se va a filtrar los datos necesarios).
    salida: .csv final (columnas útiles para la ejecución del proyecto).
    :return: Archivo csv con el filtro realizado
    """

    # Intento de lectura del archivo CSV
    try:
        df = pd.read_csv(entrada)
    except FileNotFoundError:  # Archivo no encontrado
        print("Archivo no encontrado:", entrada)  # Mensaje de error
        return None
    except Exception as e:  # Captura de otro tipo de errores
        print("Error al leer el CSV:", e)  # Mensaje de error
        return None

    print(f"Registros totales antes de la limpieza: {len(df)}\n")
    
    # Búsqueda y validación de las columnas a usar
    columnasAFiltrar = {"family", "sales", "date"}

    # Valida que las columnas se encuentren en el archivo
    if not columnasAFiltrar.issubset(df.columns):  
        # issubset() se encarga de buscar si cada columna necesaria se encuentra en el archivo
        print("El archivo no contiene las columnas deseadas: ", columnasAFiltrar)  
        return None

    # Busca en la columna "family" todos los registros que tengan el valor "BEVERAGES"
    beverages = df[df["family"] == "BEVERAGES"]  # Filtra únicamente el grupo BEVERAGES
    if "onpromotion" in beverages.columns and "id" in beverages.columns and "store_nbr" in beverages.columns:
        # Busca las columnas onpromotion, id, store_nbr en los datos filtrados beverages
        beverages = beverages.drop(columns = ["onpromotion", "id", "store_nbr"]) # Elimina la columna onpromotion con drop()
    
    # Guardar los datos filtrados en datos_filtrados.csv
    try:
        beverages.to_csv(salida, index=False)  # Guarda los datos filtrados en un nuevo CSV
    except Exception as e:  # Captura cualquier tipo de error
        print("Error al guardar el archivo.", e)
        return None

    print(f"Registros restantes después de la limpieza: {len(beverages)}")

    return beverages  # Regresa el dataframe filtrado


# Ejecutar la función
filtrarDatos()

Archivo no encontrado: train.csv


#### 3. Construcción de datos semanales

In [5]:
datos_entrada = "beverages.csv"
datos_filtrados = "ventasSemanales.csv"

def agruparVentasSemanales(entrada=datos_entrada, salida=datos_filtrados):
    """
    Lee un CSV con columnas 'date' y 'sales', convierte la fecha a datetime
    y suma las ventas por semana calendario (Lunes a Domingo). Genera un nuevo CSV con las
    ventas semanales y un índice de semana t.

    param:
    entrada: .csv inicial (dataset filtrado previamente con la familia 'BEVERAGES').
    salida: .csv final (datos útiles para la ejecución del proyecto).
    :return: Archivo csv con la agregación realizada
    """
    # Intento de lectura del archivo CSV
    try:
        df = pd.read_csv(entrada)
    except FileNotFoundError:  # Archivo no encontrado
        print("Archivo no encontrado:", entrada)  # Mensaje de error
        return None
    except Exception as e:  # Captura de otro tipo de errores
        print("Error al leer el CSV:", e)  # Mensaje de error
        return None

    print(f"Registros diarios: {len(df)}\n")

    #Verificar columnas necesarias para la agrupación
    columnasNecesarias = {"date", "sales"}
    if not columnasNecesarias.issubset(df.columns):
        print(f"Error: el archivo no contiene las columnas necesarias {columnasNecesarias} para la agrupación")
        return None

    # Convertir la columna date a datetime
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

    # Asegurar que sales sea una columna numérica
    df["sales"] = pd.to_numeric(df["sales"], errors="coerce")

    # Ordenarmos por fecha
    df = df.sort_values("date")

    # Usamos la fecha como índice para hacer resample por semana
    df = df.set_index("date")

    # Sumamos las ventas por semana 
    ventasSemanales = df["sales"].resample("W-SUN").sum().reset_index()

    #Renombramos la columna date por week_end y sales por sales_week
    ventasSemanales.rename(columns={"date":"week_end","sales":"sales_week"}, inplace = True)

    print(f"Semanas generadas: {len(ventasSemanales)}")

    # Guardar los datos filtrados en ventasSemanales.csv
    try:
        ventasSemanales.to_csv(salida, index=False)  # Guarda los datos filtrados en un nuevo CSV
    except Exception as e:  # Captura cualquier tipo de error
        print("Error al guardar el archivo.", e)
        return None

    return ventasSemanales

agruparVentasSemanales()

Registros diarios: 90936

Semanas generadas: 242


,week_end,sales_week
0,2013-01-06,343176.0
1,2013-01-13,395261.0
2,2013-01-20,398667.0
3,2013-01-27,363118.0
4,2013-02-03,379430.0
...,...,...
237,2017-07-23,1348452.0
238,2017-07-30,1361379.0
239,2017-08-06,1436505.0
240,2017-08-13,1223297.0


#### 4. Eliminación de posibles valores NaN o registros vacíos

In [6]:
datos_entrada = "ventasSemanales.csv"
datos_filtrados = "datasetFinal.csv"

def eliminacionNan(entrada=datos_entrada, salida=datos_filtrados):
    """
    Lee un archivo CSV, revisa si existen registros con valores NaN, y en caso de encontrarlos,
    elimina esos registros y devuelve un nuevo CSV.

    param:
    entrada: .csv inicial (dataset filtrado previamente con la familia 'BEVERAGES').
    salida: .csv final (datos útiles para la ejecución del proyecto).
    :return: Archivo csv sin registros con valores NaN
    """
    # Intento de lectura del archivo CSV
    try:
        df = pd.read_csv(entrada)
    except FileNotFoundError:  # Archivo no encontrado
        print("Archivo no encontrado:", entrada)  # Mensaje de error
        return None
    except Exception as e:  # Captura de otro tipo de errores
        print("Error al leer el CSV:", e)  # Mensaje de error
        return None

    print(f"Registros totales antes de la limpieza: {len(df)}\n")

    # Reemplazamos campos vacíos o con espacios por NaN
    dfReemplazado = df.replace('^\s*$', pd.NA, regex=True) # Primer parámetro = qué se va a buscar, en este caso la expresión regular ^\s*$ que significa
    # cero o más espacios en blanco.

    # Eliminar filas con algún NaN
    numRegistrosA = len(dfReemplazado)
    dfLimpio = dfReemplazado.dropna()
    numRegistrosD = len(dfLimpio)

    registrosElim = numRegistrosA - numRegistrosD

    # Mensajes informativos
    if registrosElim == 0:
        print("No se han encontrado registros vacíos o con valores NaN")
        print("El archivo se mantiene igual pero cambia el nombre a 'datasetFinal.csv'")

    print(f"Se eliminaron {registrosElim} registros con valores NaN o vacíos.")
    print(f"Registros restantes después de la limpieza: {numRegistrosD}")

     # Guardar los datos filtrados en eliminadosNan.csv
    try:
        dfLimpio.to_csv(salida, index=False)  # Guarda los datos filtrados en un nuevo CSV
    except Exception as e:  # Captura cualquier tipo de error
        print("Error al guardar el archivo.", e)
        return None

    return dfLimpio

eliminacionNan()

Registros totales antes de la limpieza: 242

No se han encontrado registros vacíos o con valores NaN
El archivo se mantiene igual pero cambia el nombre a 'datasetFinal.csv'
Se eliminaron 0 registros con valores NaN o vacíos.
Registros restantes después de la limpieza: 242


,week_end,sales_week
0,2013-01-06,343176.0
1,2013-01-13,395261.0
2,2013-01-20,398667.0
3,2013-01-27,363118.0
4,2013-02-03,379430.0
...,...,...
237,2017-07-23,1348452.0
238,2017-07-30,1361379.0
239,2017-08-06,1436505.0
240,2017-08-13,1223297.0
